In [ ]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

In [ ]:
# Path of the file to read
xiao_filepath = "/kaggle/input/dataxiao/Qa raw data.csv"

# Read the file into a variable fifa_data
xiao_data = pd.read_csv(xiao_filepath, delimiter=";")

In [ ]:
print(xiao_data)

In [ ]:
column_name = input("Enter column name: ")

In [ ]:
sns.scatterplot(x=xiao_data["Time (s)"], y=xiao_data[column_name])

In [ ]:
# Get all the column names
column_names = xiao_data.columns
columns_to_convert = column_names.tolist()

for column in columns_to_convert:
    if xiao_data[column].dtype == 'object':
        xiao_data[column] = pd.to_numeric(xiao_data[column].str.replace(',', '.'), errors='coerce')



print(xiao_data.dtypes)

In [ ]:
# create list of column names containing ".2"
column_names = [col for col in xiao_data.columns if ".2" in col]

# loop through column names and create new columns with averaged values
for name in column_names:
    # get index of current column
    idx = xiao_data.columns.get_loc(name)
    
    # get indices of preceding and following columns
    prev_idx = idx - 1
    prev_prev_idx = idx -2
    
    # create new column name
    new_col_name = name.replace(".2", "") + "_avg"
    
    # compute average of current column and its neighbors
    new_col_data = xiao_data.iloc[:, [prev_idx, idx, prev_prev_idx]].select_dtypes(include=np.number).mean(axis=1)
    
    # add new column to dataframe
    xiao_data[new_col_name] = new_col_data

In [ ]:
xiao_data.columns

In [ ]:
print(xiao_data)

In [ ]:
new_column_name = input("(you can choose _avg now)Enter column name: ")

In [ ]:
xiao_data.iloc[:, 0] = xiao_data.iloc[:, 0].astype(float)

In [ ]:
xiao_data.head()

In [ ]:
def get_mutant_index(df, col_name):
    # Get the column index for the specified column name
    col_index = df.columns.get_loc(col_name)
    
    
    
    return col_index
mutant_index_0 = get_mutant_index(xiao_data, column_name)
mutant_index_1=get_mutant_index(xiao_data,new_column_name)
sns.scatterplot(x='Time (s)', y=xiao_data.columns[mutant_index_0], data=xiao_data)
sns.scatterplot(x='Time (s)', y=xiao_data.columns[mutant_index_0 + 1], data=xiao_data)
sns.scatterplot(x='Time (s)', y=xiao_data.columns[mutant_index_0 + 2], data=xiao_data)

#sns.regplot(x='Time', y='xw11adcmu', data=xiao_data)

In [ ]:
sns.scatterplot(x='Time (s)', y=xiao_data.columns[mutant_index_0], data=xiao_data)
sns.scatterplot(x='Time (s)', y=xiao_data.columns[mutant_index_0 + 1], data=xiao_data)
sns.scatterplot(x='Time (s)', y=xiao_data.columns[mutant_index_0 + 2], data=xiao_data)
sns.scatterplot(x='Time (s)', y=xiao_data.columns[mutant_index_1], data=xiao_data)
#sns.regplot(x='Time', y='xw11adcmu', data=xiao_data)

In [ ]:
averages = []
max_values = {}
min_values = {}

for column in xiao_data.columns:
    # Extract the first 4 values from the column as a string
    first_four = xiao_data[column].astype(str).str[:4]
    all_but_four = xiao_data[column].astype(str).str[4:]
    min_but_4 = pd.to_numeric(all_but_four, errors='coerce').min()

    # Calculate the max value and min value in the column
    max_value = xiao_data[column].max()
    min_value = xiao_data[column].min()

    # Save the max and min values in the dictionaries
    max_values[column] = max_value
    min_values[column] = min_value

    # Convert the first four values to numeric and calculate the mean
    mean_of_four = pd.to_numeric(first_four, errors='coerce').mean()

    # Append the mean to the list of averages
    averages.append(mean_of_four)

# Create a string with the averages, max values, and min values for each column
output_list = [f"{col}: avg={avg:.2f}, max={max_values[col]}, min={min_values[col]}" for col, avg in zip(xiao_data.columns, averages)]
output_string = ", ".join(output_list)

# Print the output string
print(output_string)

# Create dictionaries with the averages and the min values of the first 4 values for each column
averages_dict = {col: avg for col, avg in zip(xiao_data.columns, averages)}
min_values_dict = {col: min_val for col, min_val in zip(xiao_data.columns, min_values)}

In [ ]:
min_values_dict = {column: float(min(xiao_data[column])) for column in xiao_data.columns[1:]}
max_values_dict = {column: float(max(xiao_data[column])) for column in xiao_data.columns[1:]}

# Subtract the minimum of each column
xiao_data_norm = xiao_data.copy()
xiao_data_norm[xiao_data.columns[1:]] = xiao_data[xiao_data.columns[1:]].subtract({k: v for k, v in min_values_dict.items()})

# Divide each column by its maximum-minimum
xiao_data_norm[xiao_data.columns[1:]] = xiao_data_norm[xiao_data.columns[1:]].divide({k: max_values_dict[k]-min_values_dict[k] for k in max_values_dict})

# Print the normalized data
print(xiao_data_norm)

In [ ]:
# Save the normalized data to a new CSV file in /kaggle/working/
xiao_data_norm.to_csv('/kaggle/working/normalized_data.csv', index=False)

In [ ]:
print(xiao_data_norm[new_column_name])

In [ ]:
# Create the scatterplot with normalized y-axis values
sns.scatterplot(x=xiao_data["Time (s)"].iloc[4:], y=xiao_data_norm[new_column_name].iloc[4:])

# Set the x-axis scale to logarithmic
plt.xscale("log")

# Show the plot
plt.show()

In [ ]:
def get_time_and_index_for_mutant(mutant_col):
    # Subset the data to only the specified mutant column and time
    mutant_df = xiao_data_norm[[mutant_col, "Time (s)"]]
    
    # Find the index where the mutant achieves y=1
    index_of_interest = (mutant_df[mutant_col] == 1).idxmax()
    
    # Get the corresponding time value and index
    time_of_interest = xiao_data_norm.loc[index_of_interest, "Time (s)"]
    
    return time_of_interest, index_of_interest
def get_mutant_index(df, col_name):
    # Get the column index for the specified column name
    col_index = df.columns.get_loc(col_name)
    
    
    
    return col_index
mutant_index = get_mutant_index(xiao_data_norm, new_column_name)

time_for_mutant, index_for_mutant = get_time_and_index_for_mutant(new_column_name)
print(mutant_index)
print(time_for_mutant)
print(index_for_mutant)

In [ ]:
# Create the scatterplot
sns.scatterplot(x=xiao_data_norm["Time (s)"].iloc[index_for_mutant:], y=xiao_data_norm[new_column_name].iloc[4:])

# Set the x-axis scale to logarithmic
plt.xscale("log")

# Show the plot
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Define the 3-parameter exponential decay function
def exponential_decay(x, A1, A2, A3, K1, K2, K3):
    return A1 * np.exp(-K1 * (x - time_for_mutant)) + A2 * np.exp(-K2 * (x - time_for_mutant)) + A3 * np.exp(-K3 * (x - time_for_mutant))

# Set the initial guess values for the parameters and the maximum number of function calls
p0 = [0.3, 0.3, 0.3, 0.1, 1, 10]
maxfev = 100000

# Get the data
x = xiao_data_norm["Time (s)"].iloc[index_for_mutant:]
y = xiao_data_norm[new_column_name].iloc[index_for_mutant:]

# Fit the exponential decay function to the data
popt, pcov = curve_fit(exponential_decay, x, y, p0=p0, maxfev=maxfev)

# Calculate R^2
residuals = y - exponential_decay(x, *popt)
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y - np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)

# Print the estimated parameters and R^2
print("A1= {:.3f}, A2= {:.3f}, A3= {:.3f}, K1= {:.3f}, K2= {:.3f}, K3= {:.3f}, R^2= {:.3f}".format(*popt, r_squared))

from prettytable import PrettyTable

half_life1 = np.log2(2)/popt[3]
half_life2 = np.log2(2)/popt[4]
half_life3 = np.log2(2)/popt[5]
val_at_half_life1 = exponential_decay(half_life1, *popt)
val_at_half_life2 = exponential_decay(half_life2, *popt)
val_at_half_life3 = exponential_decay(half_life3, *popt)

table = PrettyTable()
table.field_names = ["Half-life", "Exponential Value at Half-life"]

table.add_row([f"Half-life 3 ({half_life1:.6f}) associated to K1 and A1", f"{val_at_half_life1:.6f}" + ('*' if val_at_half_life1 < 0 else '')])
table.add_row([f"Half-life 2 ({half_life2:.6f}) associated to K2 and A2", f"{val_at_half_life2:.6f}" + ('*' if val_at_half_life2 < 0 else '')])
table.add_row([f"Half-life 1 ({half_life3:.6f}) associated to K3 and A3", f"{val_at_half_life3:.6f}" + ('*' if val_at_half_life3 < 0 else '')])

print(table)



# Create a new time vector starting from the end of the actual data
x_pred = np.linspace(x.iloc[-1], 100, 1000)

# Predict the values for the new time vector
y_pred = exponential_decay(x_pred, *popt)

# Create a DataFrame with the predicted values
pred_df = pd.DataFrame({'Time (s)': x_pred, new_column_name: y_pred})

# Plot the data and the fitted curve
fig, ax = plt.subplots()
ax.plot(x, y, 'bo', label='data')
ax.plot(x, exponential_decay(x, *popt), 'r-', label='fit')
plt.xscale("log")
# Set the y-axis limits to focus on the range where the half-life occurs
ax.set_ylim(bottom=-0.1, top=3.6)

# Add vertical lines for the half-life values
ax.axvline(x=half_life3, linestyle='--', color='gray')
ax.axvline(x=half_life2, linestyle='--', color='gray')
ax.axvline(x=half_life1, linestyle='--', color='gray')
# Plot the predicted values with a different color
ax.plot(pred_df['Time (s)'], pred_df[new_column_name], color='green', linestyle='-')
# Define the x range for the plots
x_pre = np.linspace(x.iloc[-1], 100, 1000)
y_pre = exponential_decay(x_pre, *popt)

# Define the individual components of the function
def component1(x):
    return popt[0] * np.exp(-popt[3] * (x - time_for_mutant))
def component2(x):
    return popt[1] * np.exp(-popt[4] * (x - time_for_mutant))
def component3(x):
    return popt[2] * np.exp(-popt[5] * (x - time_for_mutant))

# Plot the individual components of the function
fig, (ax1) = plt.subplots()
ax1.plot(x, exponential_decay(x, *popt), 'r-', label='fit')
#ax1.plot(x_pred, y_pred, 'r-', label='fit')
ax1.plot(pred_df['Time (s)'], pred_df[new_column_name], color='green', linestyle='-')
ax1.fill_between(x, component2(x)+component1(x),exponential_decay(x, *popt), where=(x >=0) & (x <= 100), alpha=0.5, color='red', label='A3')
ax1.plot(x, component2(x)+component1(x), 'orange', label='A1exp+A2exp')
ax1.fill_between(x, component1(x),component2(x)+component1(x), where=(x >=0) & (x <= 100), alpha=0.5, color='orange', label='A2')
ax1.plot(x, component1(x), 'yellow', label='A1exp')
ax1.fill_between(x, 0,component1(x), where=(x >=0) & (x <= 100), alpha=0.5, color='yellow', label='A1')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Exponential Decay')
plt.xscale("log")
ax1.legend()

# Add vertical lines for the half-life values
ax1.axvline(x=half_life3, linestyle='--', color='gray')
ax1.axvline(x=half_life2, linestyle='--', color='gray')
ax1.axvline(x=half_life1, linestyle='--', color='gray')

# Add a legend to the plot
ax1.legend()


plt.show()

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Define the 3-parameter exponential decay function
def exponentialhyp_decay(x, A1, A2, A3, K1, K2, K3):
    return A1 * np.exp(-K1 * (x - time_for_mutant)) + A2 * np.exp(-K2 * (x - time_for_mutant)) + A3 * abs(1 / (1 + K3 * (x - time_for_mutant)))

# Set the initial guess values for the parameters and the maximum number of function calls
p0 = [0.189,  0.256, 0.552, 0.638, 193.822, 2451.471]
maxfev = 10000

# Get the data
x = xiao_data_norm["Time (s)"].iloc[index_for_mutant:]
y = xiao_data_norm[new_column_name].iloc[index_for_mutant:]

# Fit the exponential decay function to the data
popt, pcov = curve_fit(exponential_decay, x, y, p0=p0, maxfev=maxfev)

# Calculate R^2
residuals = y - exponential_decay(x, *popt)
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y - np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)

# Print the estimated parameters and R^2
print("A1= {:.3f}, A2= {:.3f}, A3= {:.3f}, K1= {:.3f}, K2= {:.3f}, K3= {:.3f}, R^2= {:.3f}".format(*popt, r_squared))

from prettytable import PrettyTable

half_life1 = np.log2(2)/popt[3]
half_life2 = np.log2(2)/popt[4]
half_life3 = np.log2(2)/popt[5]
val_at_half_life1 = exponential_decay(half_life1, *popt)
val_at_half_life2 = exponential_decay(half_life2, *popt)
val_at_half_life3 = exponential_decay(half_life3, *popt)

table = PrettyTable()
table.field_names = ["Half-life", "Exponential Value at Half-life"]

table.add_row([f"Half-life 3 ({half_life1:.6f}) associated to K1 and A1", f"{val_at_half_life1:.6f}" + ('*' if val_at_half_life1 < 0 else '')])
table.add_row([f"Half-life 2 ({half_life2:.6f}) associated to K2 and A2", f"{val_at_half_life2:.6f}" + ('*' if val_at_half_life2 < 0 else '')])
table.add_row([f"Half-life 1 ({half_life3:.6f}) associated to K3 and A3", f"{val_at_half_life3:.6f}" + ('*' if val_at_half_life3 < 0 else '')])

print(table)



# Create a new time vector starting from the end of the actual data
x_pred = np.linspace(x.iloc[-1], 100, 1000)

# Predict the values for the new time vector
y_pred = exponentialhyp_decay(x_pred, *popt)

# Create a DataFrame with the predicted values
pred_df = pd.DataFrame({'Time (s)': x_pred, new_column_name: y_pred})

# Plot the data and the fitted curve
fig, ax = plt.subplots()
ax.plot(x, y, 'bo', label='data')
ax.plot(x, exponentialhyp_decay(x, *popt), 'r-', label='fit')
plt.xscale("log")
# Set the y-axis limits to focus on the range where the half-life occurs
ax.set_ylim(bottom=-0.1, top=1.1)

# Add vertical lines for the half-life values
ax.axvline(x=half_life3, linestyle='--', color='gray')
ax.axvline(x=half_life2, linestyle='--', color='gray')
ax.axvline(x=half_life1, linestyle='--', color='gray')
# Plot the predicted values with a different color
ax.plot(pred_df['Time (s)'], pred_df[new_column_name], color='green', linestyle='-')


# Define the x range for the plots
x_pre = np.linspace(x.iloc[-1], 100, 1000)
y_pre = exponentialhyp_decay(x_pre, *popt)

# Define the individual components of the function
def component1(x):
    return popt[0] * np.exp(-popt[3] * (x - time_for_mutant))
def component2(x):
    return popt[1] * np.exp(-popt[4] * (x - time_for_mutant))
def component3(x):
    return popt[2] * abs(1 / (1 + popt[5] * (x - time_for_mutant)))

# Plot the individual components of the function
fig, (ax1) = plt.subplots()
ax1.plot(x, exponentialhyp_decay(x, *popt), 'r-', label='fit')
#ax1.plot(x_pred, y_pred, 'r-', label='fit')
ax1.plot(pred_df['Time (s)'], pred_df[new_column_name], color='green', linestyle='-')
ax1.fill_between(x, component2(x)+component1(x),exponentialhyp_decay(x, *popt), where=(x >=0) & (x <= 100), alpha=0.5, color='red', label='A3')
ax1.plot(x, component2(x)+component1(x), 'orange', label='A1exp+A2exp')
ax1.fill_between(x, component1(x),component2(x)+component1(x), where=(x >=0) & (x <= 100), alpha=0.5, color='orange', label='A2')
ax1.plot(x, component1(x), 'yellow', label='hyperbolic')
ax1.fill_between(x, 0,component1(x), where=(x >=0) & (x <= 100), alpha=0.5, color='yellow', label='A1')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Exponential Decay')
plt.xscale("log")
ax1.legend()

# Add vertical lines for the half-life values
ax1.axvline(x=half_life3, linestyle='--', color='gray')
ax1.axvline(x=half_life2, linestyle='--', color='gray')
ax1.axvline(x=half_life1, linestyle='--', color='gray')

# Add a legend to the plot
ax1.legend()


plt.show()

In [ ]:
column_names_norm = xiao_data_norm.columns
columns_to_convert_norm = column_names_norm.tolist()

In [ ]:
# Ask the user if they want to plot graphs for all columns
user_input = input("Do you want to plot all the exponential 3 decay graphs for all columns? (yes/no): ")

if user_input.lower() == 'yes':
    # Initialize a PrettyTable for the summary table
    summary_table = PrettyTable()
    summary_table.field_names = ['Column Name', 'Model 1', 'Model 2']

    # Plot graphs for all columns
    for column_name in columns_to_convert_norm:
        if column_name not in ['new_column_name', "Time (s)"]:  # Skip the first column
            # Get the data
            time_for_column, index_for_column = get_time_and_index_for_mutant(column_name)
            x = xiao_data_norm["Time (s)"].iloc[index_for_column:]
            y = xiao_data_norm[column_name].iloc[index_for_column:]

            # Fit the exponential decay function to the data
            popt, pcov = curve_fit(exponential_decay, x, y, p0=p0, maxfev=maxfev)

            # Calculate R^2
            residuals = y - exponential_decay(x, *popt)
            ss_res = np.sum(residuals**2)
            ss_tot = np.sum((y - np.mean(y))**2)
            r_squared = 1 - (ss_res / ss_tot)

            # Print the estimated parameters and R^2
            print("A1= {:.3f}, A2= {:.3f}, A3= {:.3f}, K1= {:.3f}, K2= {:.3f}, K3= {:.3f}, R^2= {:.3f}".format(*popt, r_squared))

            from prettytable import PrettyTable

            half_life1 = np.log2(2)/popt[3]
            half_life2 = np.log2(2)/popt[4]
            half_life3 = np.log2(2)/popt[5]
            val_at_half_life1 = exponential_decay(half_life1, *popt)
            val_at_half_life2 = exponential_decay(half_life2, *popt)
            val_at_half_life3 = exponential_decay(half_life3, *popt)

            table = PrettyTable()
            table.field_names = ["Half-life", "Exponential Value at Half-life"]

            table.add_row([f"Half-life 3 ({half_life1:.6f}) associated to K1 and A1", f"{val_at_half_life1:.6f}" + ('*' if val_at_half_life1 < 0 else '')])
            table.add_row([f"Half-life 2 ({half_life2:.6f}) associated to K2 and A2", f"{val_at_half_life2:.6f}" + ('*' if val_at_half_life2 < 0 else '')])
            table.add_row([f"Half-life 1 ({half_life3:.6f}) associated to K3 and A3", f"{val_at_half_life3:.6f}" + ('*' if val_at_half_life3 < 0 else '')])

            

            # Create a new time vector starting from the end of the actual data
            x_pred = np.linspace(x.iloc[-1], 100, 1000)

            # Predict the values for the new time vector
            y_pred = exponential_decay(x_pred, *popt)

            # Create a DataFrame with the predicted values
            pred_df = pd.DataFrame({'Time (s)': x_pred, column_name: y_pred})

            # Plot the data and the fitted curve
            fig, ax = plt.subplots()
            ax.plot(x, y, 'bo', label='data')
            ax.plot(x, exponential_decay(x, *popt), 'r-', label='fit')
            plt.xscale("log")
            # Set the y-axis limits to focus on the range where the half-life occurs
            ax.set_ylim(bottom=-0.1, top=1.1)

            # Add vertical lines for the half-life values
            ax.axvline(x=half_life3, linestyle='--', color='gray')
            ax.axvline(x=half_life2, linestyle='--', color='gray')
            ax.axvline(x=half_life1, linestyle='--', color='gray')
            # Plot the predicted values with a different color
            ax.plot(pred_df['Time (s)'], pred_df[column_name], color='green', linestyle='-')

            # Define the x range for the plots
            x_pre = np.linspace(x.iloc[-1], 100, 1000)
            y_pre = exponential_decay(x_pre, *popt)

            # Define the individual components of the function
            def component1(x):
                return popt[0] * np.exp(-popt[3] * (x - time_for_mutant))

            def component2(x):
                return popt[1] * np.exp(-popt[4] * (x - time_for_mutant))

            def component3(x):
                return popt[2] * np.exp(-popt[5] * (x - time_for_mutant))


            # Plot the individual components of the function
            fig, (ax1) = plt.subplots()
            ax1.plot(x, exponential_decay(x, *popt), 'r-', label='fit')
            #ax1.plot(x_pred, y_pred, 'r-', label='fit')
            ax1.plot(pred_df['Time (s)'], pred_df[column_name], color='green', linestyle='-')
            ax1.fill_between(x, component2(x)+component1(x),exponential_decay(x, *popt), where=(x >=0) & (x <= 100), alpha=0.5, color='red', label='A3')
            ax1.plot(x, component2(x)+component1(x), 'orange', label='A1exp+A2exp')
            ax1.fill_between(x, component1(x),component2(x)+component1(x), where=(x >=0) & (x <= 100), alpha=0.5, color='orange', label='A2')
            ax1.plot(x, component1(x), 'yellow', label='A1exp')
            ax1.fill_between(x, 0,component1(x), where=(x >=0) & (x <= 100), alpha=0.5, color='yellow', label='A1')
            ax1.set_xlabel('Time (s)')
            ax1.set_ylabel('Exponential Decay')
            plt.xscale("log")
            ax1.legend()

            # Add vertical lines for the half-life values
            ax1.axvline(x=half_life3, linestyle='--', color='gray')
            ax1.axvline(x=half_life2, linestyle='--', color='gray')
            ax1.axvline(x=half_life1, linestyle='--', color='gray')

            # Add a legend to the plot
            ax1.legend()

            
            # Print the table with column name
            print(f"Column Name: {column_name}")
            print(table)

            # Show the plot with column name
            plt.title(f"Graph of exp3 for Column: {column_name}")
            plt.show()
else:
    print("No additional graphs will be plotted.")

In [ ]:
# Ask the user if they want to plot graphs for all columns
user_input = input("Do you want to plot all the hyperbolic exp 2 graphs for all columns? (yes/no): ")

if user_input.lower() == 'yes':
    # Initialize a PrettyTable for the summary table
    summary_table = PrettyTable()
    summary_table.field_names = ['Column Name', 'Model 1', 'Model 2']

    # Plot graphs for all columns
    for column_name in columns_to_convert_norm:
        if column_name not in ['new_column_name', "Time (s)"]:  # Skip the first column
            # Get the data
            time_for_column, index_for_column = get_time_and_index_for_mutant(column_name)
            x = xiao_data_norm["Time (s)"].iloc[index_for_column:]
            y = xiao_data_norm[column_name].iloc[index_for_column:]

            # Fit the exponential decay function to the data
            popt, pcov = curve_fit(exponentialhyp_decay, x, y, p0=p0, maxfev=maxfev)

            # Calculate R^2
            residuals = y - exponentialhyp_decay(x, *popt)
            ss_res = np.sum(residuals**2)
            ss_tot = np.sum((y - np.mean(y))**2)
            r_squared = 1 - (ss_res / ss_tot)

            # Print the estimated parameters and R^2
            print("A1= {:.3f}, A2= {:.3f}, A3= {:.3f}, K1= {:.3f}, K2= {:.3f}, K3= {:.3f}, R^2= {:.3f}".format(*popt, r_squared))

            from prettytable import PrettyTable

            half_life1 = np.log2(2)/popt[3]
            half_life2 = np.log2(2)/popt[4]
            half_life3 = 1/popt[5]
            val_at_half_life1 = exponentialhyp_decay(half_life1, *popt)
            val_at_half_life2 = exponentialhyp_decay(half_life2, *popt)
            val_at_half_life3 = exponentialhyp_decay(half_life3, *popt)

            table = PrettyTable()
            table.field_names = ["Half-life", "Exponential Value at Half-life"]

            table.add_row([f"Half-life 3 ({half_life1:.6f}) associated to K1 and A1", f"{val_at_half_life1:.6f}" + ('*' if val_at_half_life1 < 0 else '')])
            table.add_row([f"Half-life 2 ({half_life2:.6f}) associated to K2 and A2", f"{val_at_half_life2:.6f}" + ('*' if val_at_half_life2 < 0 else '')])
            table.add_row([f"Half-life 1 ({half_life3:.6f}) associated to K3 and A3", f"{val_at_half_life3:.6f}" + ('*' if val_at_half_life3 < 0 else '')])

            

            # Create a new time vector starting from the end of the actual data
            x_pred = np.linspace(x.iloc[-1], 100, 1000)

            # Predict the values for the new time vector
            y_pred = exponentialhyp_decay(x_pred, *popt)

            # Create a DataFrame with the predicted values
            pred_df = pd.DataFrame({'Time (s)': x_pred, column_name: y_pred})

            # Plot the data and the fitted curve
            fig, ax = plt.subplots()
            ax.plot(x, y, 'bo', label='data')
            ax.plot(x, exponentialhyp_decay(x, *popt), 'r-', label='fit')
            plt.xscale("log")
            # Set the y-axis limits to focus on the range where the half-life occurs
            ax.set_ylim(bottom=-0.1, top=1.1)

            # Add vertical lines for the half-life values
            ax.axvline(x=half_life3, linestyle='--', color='gray')
            ax.axvline(x=half_life2, linestyle='--', color='gray')
            ax.axvline(x=half_life1, linestyle='--', color='gray')
            # Plot the predicted values with a different color
            ax.plot(pred_df['Time (s)'], pred_df[column_name], color='green', linestyle='-')

            # Define the x range for the plots
            x_pre = np.linspace(x.iloc[-1], 100, 1000)
            y_pre = exponentialhyp_decay(x_pre, *popt)

            # Define the individual components of the function
            def component1(x):
                return popt[0] * np.exp(-popt[3] * (x - time_for_mutant))

            def component2(x):
                return popt[1] * np.exp(-popt[4] * (x - time_for_mutant))

            def component3(x):
                return popt[2] * abs(1 / (1 + popt[5] * (x - time_for_mutant)))

            # Plot the individual components of the function
            fig, (ax1) = plt.subplots()
            ax1.plot(x, exponentialhyp_decay(x, *popt), 'r-', label='fit')
            #ax1.plot(x_pred, y_pred, 'r-', label='fit')
            ax1.plot(pred_df['Time (s)'], pred_df[column_name], color='green', linestyle='-')
            ax1.fill_between(x, component2(x)+component1(x),exponentialhyp_decay(x, *popt), where=(x >=0) & (x <= 100), alpha=0.5, color='red', label='A3')
            ax1.plot(x, component2(x)+component1(x), 'orange', label='A1exp+A2exp')
            ax1.fill_between(x, component1(x),component2(x)+component1(x), where=(x >=0) & (x <= 100), alpha=0.5, color='orange', label='A2')
            ax1.plot(x, component1(x), 'yellow', label='hyperbolic')
            ax1.fill_between(x, 0,component1(x), where=(x >=0) & (x <= 100), alpha=0.5, color='yellow', label='A1')
            ax1.set_xlabel('Time (s)')
            ax1.set_ylabel('Exponential Decay')
            plt.xscale("log")
            ax1.legend()

            # Add vertical lines for the half-life values
            ax1.axvline(x=half_life3, linestyle='--', color='gray')
            ax1.axvline(x=half_life2, linestyle='--', color='gray')
            ax1.axvline(x=half_life1, linestyle='--', color='gray')

            # Add a legend to the plot
            ax1.legend()

            
            # Print the table with column name
            print(f"Column Name: {column_name}")
            print(table)

            # Show the plot with column name
            plt.title(f"Graph of hyp+exp2 for Column: {column_name}")
            plt.show()
else:
    print("No additional graphs will be plotted.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial import legendre

# Ask the user for the number of Legendre polynomials to plot
num_polynomials = int(input("Enter the number of Legendre polynomials to plot: "))

# Define the range for the dummy variable t
t_min = -1
t_max = 1

# Create a new DataFrame with modified "Time (s)" column
new_df = pd.DataFrame()
new_df["Time (s)"] = xiao_data_norm["Time (s)"].iloc[index_for_mutant:]  / 25 - 1
new_df[new_column_name] = xiao_data_norm[new_column_name].iloc[index_for_mutant:] 

# Plot the original Legendre polynomials with dummy variable t
t = np.linspace(t_min, t_max, 100)

fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# Plot the Legendre polynomials
axs[0].set_xlabel('t')
axs[0].set_ylabel('Legendre Polynomial')
axs[0].set_title('Legendre Polynomials')
for i in range(num_polynomials):
    legendre_poly_t = legendre.legval(t, [1 if j == i else 0 for j in range(num_polynomials)])
    axs[0].plot(t, legendre_poly_t, label='m{}'.format(i+1))
axs[0].legend()

# Plot the renormalized data along with the Legendre polynomials
axs[1].set_xlabel("t'")
axs[1].set_ylabel(new_column_name)
axs[1].set_title("Renormalized Data")
axs[1].scatter(new_df["Time (s)"], new_df[new_column_name], color='red', label='Renormalized Data')
axs[1].legend()

plt.tight_layout()
plt.show()
print(new_df)

In [ ]:
from scipy.optimize import least_squares as mini
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial import Legendre as L

# Get the data
x = xiao_data_norm["Time (s)"].iloc[index_for_mutant:]
y = xiao_data_norm[new_column_name].iloc[index_for_mutant:]

def myfitfun(x, y, order):
    p = L.fit(x, y, order)
    return p(x)

degrees = [2, 4, 8,num_polynomials]
plt.scatter(x, y, c='blue', lw=4, label='Data')

for deg in degrees:
    fitted_data = myfitfun(x, y, deg)
    plt.plot(x, fitted_data, label="Degree = %i" % deg)

plt.xlabel("Time (s)")
plt.ylabel(new_column_name)
plt.legend()
plt.show()
# Create a dictionary to store the results
results = {
    'Degree': [],
    'R-squared': [],
    'Coefficients': []
}
for deg in degrees:
    fitted_data = myfitfun(x, y, deg)
    
    # Calculate R-squared
    residuals = y - fitted_data
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r_squared = 1 - (ss_res / ss_tot)
    
    # Get the Legendre polynomial coefficients
    coeffs = L.fit(x, y, deg).coef
    
    # Store the results in the dictionary
    results['Degree'].append(deg)
    results['R-squared'].append(r_squared)
    results['Coefficients'].append(coeffs)

# Create a DataFrame from the results dictionary
df = pd.DataFrame(results)

# Display the table
print(df)
#print(df['Coefficients'])

In [ ]:

eqdeg=3 # how many exponentials
# Calculate the initial list of coefficients
coefficients = L.fit(x, y, num_polynomials).coef

new_list = [coefficients]  # List to store the coefficients

# Perform the recursive calculations
for i in range(1, eqdeg):
    prev_coeffs = new_list[-1]  # Get the previous coefficients
    new_coeffs = [prev_coeffs[j - 1]/(2*i-1) - prev_coeffs[j + 1]/(2*i+3) for j in range(1, len(prev_coeffs) - 1)]
    new_list.append(new_coeffs)

# Print the new list of coefficients
for coeffs in new_list:
    print(coeffs)